# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [2]:
df = pd.read_csv("./data/AviationData.csv", encoding="cp1252", low_memory=False)

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [3]:
df = df[df["Aircraft.Category"] == "Airplane"]
df = df[df["Amateur.Built"] == "No"]
df["Event.Date"] = pd.to_datetime(df["Event.Date"])
df = df[df["Event.Date"].dt.year >= 1983]
df

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
4149,20001214X42478,Incident,LAX83IA149B,1983-03-18,"LOS ANGELES, CA",United States,NaN,NaN,LAX,LOS ANGELES INTL,...,Unknown,NaN,NaN,NaN,NaN,588.0,VMC,Standing,Probable Cause,04-12-2014
4150,20001214X42478,Incident,LAX83IA149A,1983-03-18,"LOS ANGELES, CA",United States,NaN,NaN,LAX,LOS ANGELES INTL,...,NaN,"Singapore Airlines, Ltd.",NaN,NaN,NaN,588.0,VMC,Taxi,Probable Cause,04-12-2014
4171,20001214X42331,Accident,ATL83FA140,1983-03-20,"CROSSVILLE, TN",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,1.0,NaN,NaN,IMC,Cruise,Probable Cause,02-05-2011
4285,20001214X42672,Accident,FTW83LA177,1983-04-02,"MCKINNEY, TX",United States,NaN,NaN,TX05,AERO COUNTRY,...,Skydiving,NaN,1.0,NaN,NaN,4.0,VMC,Standing,Probable Cause,17-10-2016
5957,20001214X44248,Incident,MIA83IA210,1983-08-21,"NORFOLK, VA",United States,NaN,NaN,NaN,NaN,...,Unknown,NaN,NaN,NaN,NaN,289.0,VMC,Cruise,Probable Cause,01-02-2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88869,20221213106455,Accident,WPR23LA065,2022-12-13,"Lewistown, MT",United States,047257N,0109280W,KLWT,Lewiston Municipal Airport,...,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,14-12-2022
88873,20221215106463,Accident,ERA23LA090,2022-12-14,"San Juan, PR",United States,182724N,0066554W,SIG,FERNANDO LUIS RIBAS DOMINICCI,...,Personal,SKY WEST AVIATION INC TRUSTEE,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022
88876,20221219106475,Accident,WPR23LA069,2022-12-15,"Wichita, KS",United States,373829N,0972635W,ICT,WICHITA DWIGHT D EISENHOWER NT,...,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,19-12-2022
88877,20221219106470,Accident,ERA23LA091,2022-12-16,"Brooksville, FL",United States,282825N,0822719W,BKV,BROOKSVILLE-TAMPA BAY RGNL,...,Personal,GERBER RICHARD E,0.0,1.0,0.0,0.0,VMC,NaN,NaN,23-12-2022


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [4]:
# Convert injury columns to numeric
injury_cols = [
    "Total.Fatal.Injuries",
    "Total.Serious.Injuries",
    "Total.Minor.Injuries",
    "Total.Uninjured"
]

for col in injury_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")

# Construct Total Aboard
df["Total.Aboard"] = (
    df["Total.Fatal.Injuries"].fillna(0)
    + df["Total.Serious.Injuries"].fillna(0)
    + df["Total.Minor.Injuries"].fillna(0)
    + df["Total.Uninjured"].fillna(0)
)

# Construct fatal/serious injury fraction
df["serious_fatal_fraction"] = (
    df["Total.Fatal.Injuries"].fillna(0)
    + df["Total.Serious.Injuries"].fillna(0)
) / df["Total.Aboard"]

# Handle divide-by-zero
df["serious_fatal_fraction"] = df["serious_fatal_fraction"].replace([np.inf], np.nan)

df

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Total.Aboard,serious_fatal_fraction
4149,20001214X42478,Incident,LAX83IA149B,1983-03-18,"LOS ANGELES, CA",United States,NaN,NaN,LAX,LOS ANGELES INTL,...,NaN,NaN,NaN,588.0,VMC,Standing,Probable Cause,04-12-2014,588.0,0.0
4150,20001214X42478,Incident,LAX83IA149A,1983-03-18,"LOS ANGELES, CA",United States,NaN,NaN,LAX,LOS ANGELES INTL,...,NaN,NaN,NaN,588.0,VMC,Taxi,Probable Cause,04-12-2014,588.0,0.0
4171,20001214X42331,Accident,ATL83FA140,1983-03-20,"CROSSVILLE, TN",United States,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,IMC,Cruise,Probable Cause,02-05-2011,2.0,1.0
4285,20001214X42672,Accident,FTW83LA177,1983-04-02,"MCKINNEY, TX",United States,NaN,NaN,TX05,AERO COUNTRY,...,1.0,NaN,NaN,4.0,VMC,Standing,Probable Cause,17-10-2016,5.0,0.2
5957,20001214X44248,Incident,MIA83IA210,1983-08-21,"NORFOLK, VA",United States,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,289.0,VMC,Cruise,Probable Cause,01-02-2016,289.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88869,20221213106455,Accident,WPR23LA065,2022-12-13,"Lewistown, MT",United States,047257N,0109280W,KLWT,Lewiston Municipal Airport,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,14-12-2022,1.0,0.0
88873,20221215106463,Accident,ERA23LA090,2022-12-14,"San Juan, PR",United States,182724N,0066554W,SIG,FERNANDO LUIS RIBAS DOMINICCI,...,0.0,0.0,0.0,1.0,VMC,NaN,NaN,27-12-2022,1.0,0.0
88876,20221219106475,Accident,WPR23LA069,2022-12-15,"Wichita, KS",United States,373829N,0972635W,ICT,WICHITA DWIGHT D EISENHOWER NT,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,19-12-2022,1.0,0.0
88877,20221219106470,Accident,ERA23LA091,2022-12-16,"Brooksville, FL",United States,282825N,0822719W,BKV,BROOKSVILLE-TAMPA BAY RGNL,...,0.0,1.0,0.0,0.0,VMC,NaN,NaN,23-12-2022,1.0,1.0


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [5]:
df["Aircraft.damage"] = df["Aircraft.damage"].str.strip()

df["destroyed"] = (df["Aircraft.damage"] == "Destroyed").astype(int)

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [6]:
# test before cleaning
# print("Unique makes BEFORE cleaning:", df["Make"].nunique())
# # 1332 makes before

# caps and whitespace
df["Make"] = df["Make"].str.upper().str.strip()

# Drop nulls
df = df.dropna(subset=["Make"])
make_counts = df["Make"].value_counts()
valid_makes = make_counts[make_counts >= 50].index

df = df[df["Make"].isin(valid_makes)]

# test
# print("Unique makes AFTER cleaning:", df["Make"].nunique())
# df["Make"].value_counts().head(20)
# 32 makes after


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [7]:
# test before cleaning
# print("Unique models BEFORE cleaning:", df["Model"].nunique())
# 2034 before

# Drop nulls/NaNs
df = df.dropna(subset=["Model"])
df["Model"] = df["Model"].str.upper().str.strip()
df["Make_Model"] = df["Make"] + "_" + df["Model"]

# Inspect after cleaning
print("Unique models AFTER cleaning:", df["Model"].nunique())
df["Make_Model"].value_counts().head(10)

Unique models AFTER cleaning: 2025


Make_Model
CESSNA_172     769
BOEING_737     403
CESSNA_152     316
CESSNA_182     304
CESSNA_172S    276
PIPER_PA28     273
CESSNA_172N    249
CESSNA_180     213
CESSNA_172M    180
CESSNA_150     179
Name: count, dtype: int64

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [8]:
categorical_cols = [
    "Engine.Type",
    "Weather.Condition",
    "Broad.phase.of.flight",
    "Purpose.of.flight"
]

for col in categorical_cols:
    df[col] = df[col].str.upper().str.strip()

# Convert number of engines to numeric
df["Number.of.Engines"] = pd.to_numeric(df["Number.of.Engines"], errors="coerce")
df.loc[df["Number.of.Engines"] == 0, "Number.of.Engines"] = np.nan

# Quick inspection
for col in categorical_cols:
    print(f"\n{col} value counts:")
    print(df[col].value_counts().head())

print("\nNumber.of.Engines summary:")
print(df["Number.of.Engines"].describe())


Engine.Type value counts:
Engine.Type
RECIPROCATING    12835
TURBO PROP         931
TURBO FAN          701
UNKNOWN            105
TURBO JET           71
Name: count, dtype: int64

Weather.Condition value counts:
Weather.Condition
VMC    14295
IMC      905
UNK      262
Name: count, dtype: int64

Broad.phase.of.flight value counts:
Broad.phase.of.flight
LANDING        1110
TAKEOFF         425
CRUISE          238
APPROACH        210
MANEUVERING     127
Name: count, dtype: int64

Purpose.of.flight value counts:
Purpose.of.flight
PERSONAL              9844
INSTRUCTIONAL         2410
AERIAL APPLICATION     724
BUSINESS               409
UNKNOWN                303
Name: count, dtype: int64

Number.of.Engines summary:
count    15785.000000
mean         1.172506
std          0.414149
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          4.000000
Name: Number.of.Engines, dtype: float64


### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [9]:
non_null_percentage = df.notnull().mean()
df = df.loc[:, non_null_percentage >= 0.80]

df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 17879 entries, 4150 to 88886
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                17879 non-null  object        
 1   Investigation.Type      17879 non-null  object        
 2   Accident.Number         17879 non-null  object        
 3   Event.Date              17879 non-null  datetime64[ns]
 4   Location                17875 non-null  object        
 5   Country                 17878 non-null  object        
 6   Latitude                15981 non-null  object        
 7   Longitude               15978 non-null  object        
 8   Injury.Severity         17162 non-null  object        
 9   Aircraft.damage         16815 non-null  object        
 10  Aircraft.Category       17879 non-null  object        
 11  Registration.Number     17715 non-null  object        
 12  Make                    17879 non-null  object  

(17879, 29)

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [10]:
df.to_csv("./data/cleaned_aviation_data.csv", index=False)